# Train your own Model with Nerf Studio from Video

Author: Deeepwin  
Date: 18.03.2023 
***

## Installation

This installation instructions explains in how to install Nerfstudio in a Conda enviroment that has a separte CUDA installtion. It makes the installation of Nerfstudio a bit more tricky. In my case I have installed CUDA version 11.3 in Conda which I want to use for this project. The Conda environment is called `points`. On the Ubuntu installation I run CUDA 12.1.

### Checking CUDA Installation

In [1]:
# check CUDA versions, must be 11.3.1
!conda list cudatoolkit
!conda list cudnn
!conda list cuda-nvcc

# packages in environment at /home/martin/anaconda3/envs/points:
#
# Name                    Version                   Build  Channel
cudatoolkit               11.3.1               h2bc3f7f_2  
# packages in environment at /home/martin/anaconda3/envs/points:
#
# Name                    Version                   Build  Channel
cudnn                     8.2.1                cuda11.3_0  
nvidia-cudnn-cu11         8.5.0.96                 pypi_0    pypi
# packages in environment at /home/martin/anaconda3/envs/points:
#
# Name                    Version                   Build  Channel
cuda-nvcc                 11.3.122             h4814707_0    nvidia/label/cuda-11.3.1


In [2]:
# check Pytorch version, must be 1.13.1
torch_version = !pip show torch
torch_version[1]

'Version: 1.13.1'

Pytorch automatically installs nvidia_cublas_cu11, which will cause an error in the installation of Nerfstudio. That's why we need to remove it here.

In [ ]:
!pip uninstall nvidia_cublas_cu11

### Setup Tiny CUDA NN

Setting up `tiny-cuda-nn` is a pain and it might work a bit different on your system. 

For CUDA version 11.3 you need g++ version 9, max 10.2.1, see [here](https://stackoverflow.com/questions/6622454/cuda-incompatible-with-my-gcc-version). General instructions in how to setup different versions of g++ can be found [here](https://tuxamito.com/wiki/index.php/Installing_newer_GCC_versions_in_Ubuntu).

In [ ]:
!sudo apt -y install g++-9
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 9
!sudo update-alternatives --config g++
!g++ --version


In [ ]:
!sudo apt -y install gcc-9
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 9
!sudo update-alternatives --config gcc
!gcc --version

There is a problem with include path, adjust path to your system:

In [ ]:
!export CPATH=/home/martin/anaconda3/envs/points/lib/python3.7/site-packages/nvidia/cuda_runtime/include:$CPATH

Now set CUDA_HOME enviroment variable. Otherwise you cannot install `tiny-cuda-nn`. Update path according to your Conda installation.

In [ ]:
!conda env config vars set CUDA_HOME="/home/martin/anaconda3/envs/points/lib/"
!conda env config vars list

In [ ]:
!conda activate points

It also requires cuSPARSE library. Alo here, adjust path accordingly.

In [ ]:
!conda install -c "nvidia/label/cuda-11.3.1" libcusparse
!export CPATH=/home/martin/anaconda3/envs/points/include:$CPATH

Install `tiny-cuda-nn`

In [ ]:
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

In [3]:
# check if installation was successful
import tcnn


ModuleNotFoundError: No module named 'tcnn'

Finally install Nerfstudio see [website](https://docs.nerf.studio/en/latest/quickstart/installation.html).

In [ ]:
%%capture
!pip install nerfstudio

## Verfiy with Existing Data

In [ ]:
!ns-download-data nerfstudio --capture-name=poster

In [ ]:
!ns-train nerfacto --data data/nerfstudio/poster